In [ ]:
# coding: utf-8

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.cm import get_cmap
import scipy.optimize

# Input experimental data

In [ ]:
def load_df(i):
    df_list = []
    for name in ['Degrading', 'Improving']:
        df = pd.read_excel('experimental_data.xlsx', sheet_name = '{} {}'.format(name, i))
        df = df[df.index % 2 == 0]
        df_list.append(df)
    return df_list

df1_list = load_df(1)
df2_list = load_df(2)

# Compute Volumes

In [ ]:
p0 = 99100 # Pa
h1 = 0.092 # m
h2 = 0.346
h3 = 0.140
h4 = 0.303

v_k = 50 # cm^3

dh1 = abs(h2 - h1)
dh2 = abs(h4 - h3)

sigma_h = 0.001

rho = 885
g = 9.81

In [ ]:
p1 = rho * g * dh1
v_fv = (p0 - p1) / p1 * v_k
sigma_p1 = 2 * g * rho * sigma_h
sigma_fv = p0 / p1 ** 2 * sigma_p1 * v_k

In [ ]:
sigma_fv

In [ ]:
p2 = rho * g * dh2
v_vv = (p0 / p2) * v_k - v_fv - v_k
sigma_p2 = 2 * g * rho * sigma_h
sigma_vv = np.sqrt((p0 / p2 ** 2 * v_k * sigma_p2) ** 2 + sigma_fv ** 2)

In [ ]:
v_vv

In [ ]:
sigma_vv

# Least Squares

In [ ]:
def lstsqb(x, y):
    return (x.multiply(y).mean() - x.mean() * y.mean()) / (x.multiply(x).mean() - x.mean() ** 2)

In [ ]:
def lstsqa(x, y):
    return y.mean() - lstsqb(x, y) * x.mean()

In [ ]:
def lstsqbsem(x, y):
    return 1 / np.sqrt(len(x)) * np.sqrt((y.multiply(y).mean() - y.mean() ** 2) / (x.multiply(x).mean() - x.mean() ** 2) - lstsqb(x, y) ** 2)

# Part 1

In [ ]:
def get_min_max(series, k1 = 3 / 4, k2 = 11 / 10):
    return k1 * min(series), k2 * max(series)

In [ ]:
def plot_data_linear_scale(ax, df, p, pe):
    y_min, y_max = get_min_max(df['P, 10^-4 мм. рт. ст.'])
    x_min, x_max = get_min_max(df['t, с'])
    y_min = 0

    ax.xaxis.set_major_locator(plt.MultipleLocator(10))
    ax.yaxis.set_major_locator(plt.MultipleLocator(2))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(1.0))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.2))
    
    plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    
    ax.set_ylim(bottom = y_min, top = y_max)
    ax.set_xlim(left = x_min, right = x_max)
    
    ax.grid(visible = True, linestyle = '--', which = 'both')
    x = np.linspace(x_min, x_max, 1000)
    ax.scatter(df['t, с'], df['P, 10^-4 мм. рт. ст.'])
    
    ax.plot(x, np.exp(p[1]) * np.exp(p[0] * x) + pe)
    ax.set_xlabel('$t, с$')
    ax.set_ylabel('$P, 10^{-4} мм.рт.ст.$')

In [ ]:
def plot_data_log_scale(ax, df, pe, start):
    x_min, x_max = get_min_max(df['t, с'])
    
    y_0 = np.log(df['P, 10^-4 мм. рт. ст.'] - pe)
    y_min, y_max = get_min_max(y_0)
    y = y_0.iloc[start:]
    
    p, v = np.polyfit(df['t, с'].iloc[start:], y, deg = 1, cov = True)
    ax.xaxis.set_major_locator(plt.MultipleLocator(1))
    ax.yaxis.set_major_locator(plt.MultipleLocator(0.5))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.5))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.25))
    
    ax.set_ylim(bottom = y_min, top = y_max)
    ax.set_xlim(left = x_min, right = x_max)
    
    ax.grid(visible = True, linestyle = '--', which = 'both')
    x = np.linspace(x_min, x_max, 1000)
    ax.scatter(df['t, с'], y_0)
    err = np.sqrt(np.diag(v))
        
    ax.plot(x, x * p[0] + p[1], label = 'Линейная модель: $f(\ln{P - P_пр}) = a + b \cdot t$')
    ax.plot([], [], ' ', label = '$b = {b:.3f} \pm {db:.3f}, a = {a:.3f} \pm {da:.3f}$'.format(b = p[0], db = err[0], a = p[1], da = err[1]))
    
    ax.set_xlabel('$t, с$')
    ax.set_ylabel('$ln{P - P_{пр}}$')
    
    ax.legend()
    return p, err

In [ ]:
def calculate_least_squares(df, start, pe):
    df1 = df[start:].copy()
    
    df1['ln(P - P_пр)'] = np.log(df1['P, 10^-4 мм. рт. ст.'] - pe)
    df1['ln(P - P_пр) * t'] = df1['t, с'] * df1['ln(P - P_пр)']
    
    df1['t^2, с^2'] = df1['t, с'] * df1['t, с']
    df1['ln(P - P_пр)^2'] = df1['ln(P - P_пр)'] * df1['ln(P - P_пр)']
    
    x_m = df1['t, с'].mean()
    y_m = df1['ln(P - P_пр)'].mean()
    
    xx_m = df1['t^2, с^2'].mean()
    xy_m = df1['ln(P - P_пр) * t'].mean()
    yy_m = df1['ln(P - P_пр)^2'].mean()
    
    return df1, x_m, y_m, xx_m, xy_m, yy_m

In [ ]:
output1, x1_m, y1_m, xx1_m, xy1_m, yy1_m = calculate_least_squares(df1_list[1], 5, 1.3)
output2, x2_m, y2_m, xx2_m, xy2_m, yy2_m = calculate_least_squares(df2_list[1], 7, 1.3)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 1. Зависимость $P(t)$ в логарифмических координатах (улучшение 1)')
p1, err1 = plot_data_log_scale(ax, df1_list[1], 1.3, 5)
fig.savefig('plots/plot1.pdf')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 2. Зависимость $P(t)$ в логарифмических координатах (улучшение 2)')
p2, err2 = plot_data_log_scale(ax, df2_list[1], 1.3, 7)
fig.savefig('plots/plot2.pdf')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 3. Зависимость $P(t)$ в линейных координатах (улучшение 1)')
plot_data_linear_scale(ax, df1_list[1], p1, 1.3)
fig.savefig('plots/plot3.pdf')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 4. Зависимость $P(t)$ в линейных координатах (улучшение 2)')
plot_data_linear_scale(ax, df2_list[1], p2, 1.3)
fig.savefig('plots/plot4.pdf')

In [ ]:
v = v_fv + v_vv + v_k
sigma_v = np.sqrt(sigma_vv ** 2 + sigma_fv ** 2)

In [ ]:
w1 = -p1[0] * v_vv
w2 = -p2[0] * v_vv

In [ ]:
sigma_w1 = w1 * np.sqrt((err1[0] / p1[0]) ** 2 + (sigma_vv / v_vv) ** 2)
sigma_w2 = w2 * np.sqrt((err2[0] / p2[0]) ** 2 + (sigma_vv / v_vv) ** 2)

In [ ]:
print('w1 = {}\nw2 = {}\nsigma_w1 = {}\nsigma_w2 = {}'.format(w1, w2, sigma_w1, sigma_w2))

# Plots improving vacuum

In [ ]:
p3, v3 = np.polyfit(df1_list[0]['t, с'], df1_list[0]['P, 10^-4 мм. рт. ст.'], deg = 1, cov = True)
p4, v4= np.polyfit(df2_list[0]['t, с'], df2_list[0]['P, 10^-4 мм. рт. ст.'], deg = 1, cov = True)
err3 = np.sqrt(np.diag(v3))
err4 = np.sqrt(np.diag(v4))

In [ ]:
p3

In [ ]:
def plot_improving_linear_scale(ax, df, p, err):
    y_min, y_max = get_min_max(df['P, 10^-4 мм. рт. ст.'])
    x_min, x_max = get_min_max(df['t, с'])
    y_min = 0

    ax.xaxis.set_major_locator(plt.MultipleLocator(10))
    ax.yaxis.set_major_locator(plt.MultipleLocator(2))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(1.0))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.2))
    
    plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    
    ax.set_ylim(bottom = y_min, top = y_max)
    ax.set_xlim(left = x_min, right = x_max)
    
    ax.grid(visible = True, linestyle = '--', which = 'both')
    x = np.linspace(x_min, x_max, 1000)
    ax.scatter(df['t, с'], df['P, 10^-4 мм. рт. ст.'])
    
    ax.plot(x, x * p[0] + p[1], label = 'Линейная модель: $f(\ln{P - P_пр}) = a + b \cdot t$')
    ax.plot([], [], ' ', label = '$b = {b:.3f} \pm {db:.3f}, a = {a:.3f} \pm {da:.3f}$'.format(b = p[0], db = err[0], a = p[1], da = err[1]))
    
    ax.legend()
    ax.set_xlabel('$t, с$')
    ax.set_ylabel('$P, 10^{-4} мм.рт.ст.$')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 5. Зависимость $P(t)$ в линейных координатах (ухудшение 1)')
plot_improving_linear_scale(ax, df1_list[0], p3, err3)
fig.savefig('plots/plot5.pdf')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 6. Зависимость $P(t)$ в линейных координатах (ухудшение 2)')
plot_improving_linear_scale(ax, df2_list[0], p4, err4)
fig.savefig('plots/plot6.pdf')

In [ ]:
q_n1 = df1_list[0]['P, 10^-4 мм. рт. ст.'].iloc[0] * w1 - p3[0] * v_vv
q_n2 = df2_list[0]['P, 10^-4 мм. рт. ст.'].iloc[0] * w1 - p4[0] * v_vv

In [ ]:
sigma_qn = np.sqrt((df1_list[0]['P, 10^-4 мм. рт. ст.'].iloc[0] * sigma_w1) ** 2 + (p3[0] * sigma_vv) ** 2)

In [ ]:
sigma_qn

In [ ]:
q_n1

In [ ]:
q_n2

In [ ]:
mu = 29 # g/mol
R = 8.31
T = 300 # K

m1 = mu * q_n1 / (R * T)
m2 = mu * q_n2 / (R * T)

# Compute flow

In [ ]:
d = 0.8e-1
l = 10.8
r = d / 2

mmrt = 133.332

p1 = 1.5 * 10 ** (1) * mmrt
p2 = 1.6 * mmrt

In [ ]:
flow = 4 / 3 * r ** 3 * np.sqrt(2 * np.pi * R * T / mu) * (p1 - p2) / l

In [ ]:
flow

# Results

In [ ]:
output1.to_excel('plots/output1.xlsx')
output2.to_excel('plots/output2.xlsx')